# Elementos de PLN, o El Hacedor 2.0

Siguiendo nuestra lección anterior, optimizaremos nuestro modelo de red neuronal para crear nombres. Ahora, lo haremos en el estilo del *paper* [Bengio et. al, 2003](https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf).

In [1]:
import torch
import torch.nn.functional as F
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import os
import requests
from pathlib import Path

In [2]:
path = Path('data/')
if not path.is_dir():
  path.mkdir(parents=True, exist_ok=True)

with open(path / 'nombres.txt', 'wb') as f:
  request = requests.get('https://github.com/DanteNoguez/CalculusRatiocinator/raw/main/data/nombres.txt')
  f.write(request.content)

In [3]:
nombres = open('data/nombres.txt', 'r').read().splitlines()
nombres[:8]

['maria', 'rosa', 'jose', 'carmen', 'ana', 'juana', 'antonio', 'elena']

In [4]:
len(nombres)

21029

In [5]:
V = sorted(set(''.join(nombres)))
paf = {p:f+1 for f, p in enumerate(V)}
paf['.'] = 0
fap = {f:p for p,f in paf.items()}
print(fap)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


### Un modelo neuronal probabilístico de lenguaje

Primero, comenzaremos dividiendo nuestros datos en «bloques». Por ejemplo, en nuestro modelo de bigramas, el bloque contenía un solo carácter, puesto que realizábamos la predicción a partir de una letra; pero podemos aumentar el «contexto» de nuestras predicciones para involucrar más letras al momento de predecir la siguiente. Veamos, por ejemplo, cómo luciría nuestro tratamiento de los datos si hiciéramos bloques de tres caracteres para predecir el siguiente:

In [6]:
def construir_dataset(nombres):
  block_size = 3 # longitud del contexto
  X, Y = [], []
  for n in nombres:
    #print(f'nombre: {n}')
    contexto = [0] * block_size
    for c in n + '.':
      ix = paf[c]
      X.append(contexto)
      Y.append(ix)
      #print(''.join(fap[i] for i in contexto), '----> ', fap[ix])
      contexto = contexto[1:] + [ix]
  
  X = torch.tensor(X) # contexto
  Y = torch.tensor(Y) # objetivo
  return X, Y

```{margin}
“The training set is a sequence $w_1 · · · w_T$ of words $w_t \in V$, where the vocabulary $V$ is a large but finite set.”
```

In [18]:
construir_dataset(nombres[:3])

nombre: maria
... ---->  m
..m ---->  a
.ma ---->  r
mar ---->  i
ari ---->  a
ria ---->  .
nombre: rosa
... ---->  r
..r ---->  o
.ro ---->  s
ros ---->  a
osa ---->  .
nombre: jose
... ---->  j
..j ---->  o
.jo ---->  s
jos ---->  e
ose ---->  .


(tensor([[ 0,  0,  0],
         [ 0,  0, 13],
         [ 0, 13,  1],
         [13,  1, 18],
         [ 1, 18,  9],
         [18,  9,  1],
         [ 0,  0,  0],
         [ 0,  0, 18],
         [ 0, 18, 15],
         [18, 15, 19],
         [15, 19,  1],
         [ 0,  0,  0],
         [ 0,  0, 10],
         [ 0, 10, 15],
         [10, 15, 19],
         [15, 19,  5]]),
 tensor([13,  1, 18,  9,  1,  0, 18, 15, 19,  1,  0, 10, 15, 19,  5,  0]))

Nuestra intuición detrás de esta aproximación es que el lenguaje funciona mejor con contexto: así como el sentido de un concepto se entiende mejor en contexto, también los caracteres se pueden predecir más razonablemente dado un contexto más amplio. 

Similar a como habíamos hecho anteriormente, construimos una matriz `X` para contener el contexto como entrada y un vector `Y` que contiene el objetivo (es decir, carácter) que debe seguir a cada respectivo contexto. Como se puede apreciar, solamente estamos construyendo `X` e `Y` con sus respectivos índices del vocabulario.

Dado que solo tomamos 3 nombres como ejemplo, nuestros datos únicamente contienen 16 contextos o *inputs* y 16 objetivos o *outputs*:

In [159]:
X, Y = construir_dataset(nombres[:3])
X.shape, Y.shape

(torch.Size([16, 3]), torch.Size([16]))

Ahora estamos listos para hacer el *embedding*. Mientras que en el *paper* los datos se incrustan en una tabla de consulta de 30 dimensiones (o *features*) para un vocabulario de 17,000 palabras, nosotros —que únicamente tenemos un vocabulario de 27 caracteres— podemos aproximarnos a la incrustación con algo más pequeño, como una incrustación de dos dimensiones.

In [160]:
C = torch.randn((27, 2)) # tabla de consulta
C

tensor([[ 1.7079,  0.2444],
        [-0.9648,  0.9906],
        [ 0.3772,  0.6053],
        [ 0.5600, -0.9066],
        [ 0.0494,  1.8393],
        [-0.2774,  1.6655],
        [-1.8486, -0.8595],
        [-2.8719, -0.1518],
        [ 0.1987,  0.3194],
        [-1.0431, -0.1715],
        [-0.4272,  2.0461],
        [ 0.8884, -0.0656],
        [-1.6604,  0.4760],
        [ 0.1157,  1.4921],
        [-1.7283,  0.3410],
        [-0.9365,  0.2463],
        [ 2.4093, -0.1046],
        [ 0.0064,  1.6897],
        [-0.2445, -0.8068],
        [-0.5709,  0.2377],
        [ 0.0886,  0.0178],
        [ 0.8750,  0.5877],
        [ 0.6039, -0.5445],
        [ 0.1814, -0.7042],
        [ 1.2580,  0.7008],
        [ 0.1558,  0.9199],
        [ 0.8629,  0.9445]])

Como vemos, cada *token* o elemento del volcabulario se incrustará en dos dimensiones, es decir, tendrá dos números asociados. Ahora, tomaremos un atajo que nos permitirá ser más eficientes con la codificación y la primera capa de la red neuronal. Anteriormente, habíamos hecho un *one-hot encoding* para luego pasarlo por una capa `W`; pero, bien visto, estos dos pasos pueden ser omitidos porque consiguen el mismo resultado que la incrustación en nuestra tabla `C`. 

Primero: la codificación *one-hot*, si fuéramos a multiplicarla por `C`, anularía todos los valores de `C` al multiplicarlos por 0 y conservaría una fila correspondiente a la de la multiplicación por 1. Ergo, podemos omitir la multiplicación y hacer una indexación para asociar directamente cada carácter con cada fila que un vector *one-hot* multiplicaría por 1. Dicho esto, podemos concebir a `C` como un equivalente de la capa `W`, puesto que consiste de valores aleatorios que asignan un número a cada carácter y luego pueden optimizarse con propagación hacia atrás.

Dicho esto, la indexación (*embedding*) será bastante simple:

```{margin}
“A mapping $C$ from any element $i$ of $V$ to a real vector $C(i) \in \mathbb{R}^m$. It represents the distributed feature vectors associated with each word in the vocabulary. In practice, C is represented by a $\left|V\right| \times m$ matrix of free parameters”.
```

In [161]:
emb = C[X] # embedding

f'Segunda fila de C: {C[1].numpy()} | Tercer valor del tercer bloque incrustado (es decir, letra a): {emb[2][2].numpy()}'

'Segunda fila de C: [-0.96479154  0.99060136] | Tercer valor del tercer bloque incrustado (es decir, letra a): [-0.96479154  0.99060136]'

In [162]:
print('Dimensiones del embedding: ', emb.shape) 
print('Tres bloques del embedding, correspondientes a «..m», «.ma» y «mar»:', emb[1:4])

Dimensiones del embedding:  torch.Size([16, 3, 2])
Tres bloques del embedding, correspondientes a «..m», «.ma» y «mar»: tensor([[[ 1.7079,  0.2444],
         [ 1.7079,  0.2444],
         [ 0.1157,  1.4921]],

        [[ 1.7079,  0.2444],
         [ 0.1157,  1.4921],
         [-0.9648,  0.9906]],

        [[ 0.1157,  1.4921],
         [-0.9648,  0.9906],
         [-0.2445, -0.8068]]])


Ahora, echemos un vistazo a la arquitectura que deseamos lograr:

```{figure} ../../img/bengio2003.png
---
width: 70%
name: bengio2003
---
Arquitectura neuronal $f\left(i, w_{t-1}, \cdots, w_{t-n+1}\right)=g\left(i, C\left(w_{t-1}\right), \cdots, C\left(w_{t-n+1}\right)\right)$ donde $g$ es la red neuronal y $C(i)$ es el $i$-ésimo vector de cada palabra. En nuestro caso, utilizamos bloques de tres letras (`contexto`, vector `X`) en lugar de palabras ($w$).
```

Como vemos, tenemos casi terminado el inicio y únicamente nos falta concatenar entre sí los bloques del *embedding*, puesto que juntos atravesarán la misma capa de neuronas.
```{margin}
“\[...] $x$ is the word features layer activation vector, which is the concatenation of the input word features from the matrix $C$: $x = C\left(w_{t-1}\right), \left(w_{t-2}\right) \cdots, C\left(w_{t-n+1}\right)$.”
```
Para conseguirlo, podemos utilizar distintos métodos con PyTorch:

In [163]:
metodo1 = torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], 1)
print(f"""Ahora, en lugar de estar contenidos en bloques de tres filas: 
{emb[0]}
Estarían contenidos en bloques de una fila (seis columnas):
{metodo1[0]}""")

Ahora, en lugar de estar contenidos en bloques de tres filas: 
tensor([[1.7079, 0.2444],
        [1.7079, 0.2444],
        [1.7079, 0.2444]])
Estarían contenidos en bloques de una fila (seis columnas):
tensor([1.7079, 0.2444, 1.7079, 0.2444, 1.7079, 0.2444])


In [164]:
metodo2 = torch.cat(torch.unbind(emb, 1), 1)
f'Aunque también es equivalente: {metodo2[0]}'

'Aunque también es equivalente: tensor([1.7079, 0.2444, 1.7079, 0.2444, 1.7079, 0.2444])'

Pero el método más eficiente[^1] y simple es `view`. Como primer argumento, colocaremos `-1` para que de esta forma PyTorch infiera el tamaño de la dimensión 0 que debería tener el tensor (sería, pues, equivalente a colocar `emb[0].shape`), y como segundo argumento `6` porque queremos que el tensor tenga las 6 columnas correspondientes a un bloque de tres *tokens*:

In [165]:
emb.view(-1, 6)[0], emb.view(-1, 6).shape # esta es la variable x del paper

(tensor([1.7079, 0.2444, 1.7079, 0.2444, 1.7079, 0.2444]), torch.Size([16, 6]))

Ahora ya tenemos lo suficiente para definir más variables:

> “Let $h$ be the number of hidden units[^2], and $m$ the number of features associated with each word. When no direct connections from word features to outputs are desired, the matrix $W$ is set to 0 . The free parameters of the model are the output biases $b$ (with $|V|$ elements), the hidden layer biases $d$ (with $h$ elements), the hidden-to-output weights $U$ (a $|V| \times h$ matrix), the word features to output weights $W$ (a $|V| \times(n-1) m$ matrix), the hidden layer weights $H$ (a $h \times(n-1) m$ matrix), and the word features $C$(a $|V| \times m$ matrix $)$: $\theta=(b, d, W, U, H, C)$”.

In [166]:
f'Número de features (m), es decir, número de componentes de cada bloque: {emb.view(-1, 6).size(dim=1)} | Elementos de |V|: {len(V)+1}'

'Número de features (m), es decir, número de componentes de cada bloque: 6 | Elementos de |V|: 27'

Él número de parámetros ($h$) depende del problema a tratar: generalmente, a mayor cantidad de datos, es mejor mayor cantidad de parámetros. En general, cuestiones técnicas como esta dependen de la evaluación experimental que hagamos de nuestro modelo: tras pruebas con diferentes números de parámetros, podemos elegir la que más efectiva y eficiente sea. En el *paper*, por ejemplo, probaron con 50 y 100 *hidden units*. Dado que por el momento solo estamos ejemplificando con tres nombres, $h = 50$ unidades serán suficientes.

Por otra parte, fijar un valor de 0 a $W$ es igual que no utilizarla, de manera que omitiremos su definición porque no la necesitamos.

In [167]:
h = 50
d = torch.randn((h))
H = torch.randn((6, h))

a = torch.tanh(emb.view(-1, 6) @ H + d)
a.shape

torch.Size([16, 50])

Ahora, la capa oculta previa al *output*, es decir, la *hidden-to-output layer* se compone de $U$ y $b$, mientras que el resultado de esta capa son —el lector lo recordará— lo que llamamos *logits*, es decir, el resultado de la última capa de la red neuronal. Estos *logits* serán convertidos en  probabilidades y, con ello, tendremos el *output* de toda la red. 

In [168]:
U = torch.randn((h, 27))
b = torch.randn(27)

logits = a @ U + b
logits.shape

torch.Size([16, 27])

Ahora, para convertir esto en probabilidades, haremos lo mismo que en la lección pasada:

In [169]:
counts = logits.exp()
prob = counts / counts.sum(1, keepdims=True)
prob[0].sum()

tensor(1.)

Para crear nuestra función de pérdida, necesitamos seleccionar nuestros objetivos (con base en el índice que nos dio `Y`):

In [170]:
prob[torch.arange(16), Y]

tensor([1.5711e-06, 2.9872e-04, 5.4979e-05, 3.6278e-07, 2.1988e-07, 1.4920e-06,
        4.6494e-04, 6.1111e-05, 9.9566e-01, 4.6007e-03, 3.9694e-07, 2.1725e-07,
        1.4721e-05, 7.4690e-03, 1.1107e-10, 8.2172e-06])

Ejemplifiquemos: si nuestro *input* es «mar» (`emb[3]`), nuestra probabilidad debe ser alta para que el número del *embedding* que represente la letra «i» de «María» sea un *output*:

In [171]:
print(f"""En un inicio, cada letra de «mar» correspondía a su índice en el vocabulario: {X[3]}
Cuando pasamos estos índices a una matriz para que fueran representados por dos números, obtuvimos: {emb[3]}
Al mismo tiempo, la letra «i» fue guardada como objetivo en Y, siendo su índice: {Y[3]}
De manera que la probabilidad de nuestra red neuronal debe ser alta para el número que representa la letra «i» en el embedding: {-prob[3, Y[3]]}""")

En un inicio, cada letra de «mar» correspondía a su índice en el vocabulario: tensor([13,  1, 18])
Cuando pasamos estos índices a una matriz para que fueran representados por dos números, obtuvimos: tensor([[ 0.1157,  1.4921],
        [-0.9648,  0.9906],
        [-0.2445, -0.8068]])
Al mismo tiempo, la letra «i» fue guardada como objetivo en Y, siendo su índice: 9
De manera que la probabilidad de nuestra red neuronal debe ser alta para el número que representa la letra «i» en el embedding: -3.6277964454711764e-07


Dado que no hemos entrenado la red, la probabilidad anterior es muy baja. Para poder entrenarla, formularemos la función de pérdida mediante el promedio del logaritmo natural de las probabilidades que la red neuronal asigna a cada objetivo, y finalmente hacemos de este un número positivo al multiplicarlo por $-1$, igual que en la lección pasada:

In [172]:
perdida = -prob[torch.arange(16), Y].log().mean()
perdida

tensor(11.1475)

Usando PyTorch, podemos simplificar todo este proceso mediante el uso de `cross_entropy`:

In [173]:
F.cross_entropy(logits, Y)

tensor(11.1475)

Finalmente, agrupamos los parámetros —$\theta$— en una variable:

In [174]:
parametros = [b, d, U, H, C]

Indicamos a PyTorch que requeriremos gradientes para el entrenamiento de nuestros parámetros:

In [175]:
for p in parametros:
  p.requires_grad = True

Finalmente, podemos entrenar la red:

In [176]:
for i in range(100):
  # paso hacia delante
  emb = C[X]
  h = torch.tanh(emb.view(-1, 6) @ H + d)
  logits = h @ U + b
  perdida = F.cross_entropy(logits, Y)
  
  # propagación hacia atrás
  for p in parametros:
    p.grad = None
  perdida.backward()
  
  for p in parametros:
    p.data += -0.1 * p.grad

print(perdida.item())

0.22844049334526062


Bien, hemos conseguido un resultado decente en nuestro ejemplo. Ahora es momento de entrenar nuestra red con todos los nombres a la vez. 

Aunque primero, hacen falta algunos ajustes. En primer lugar, evitaremos que nuestro modelo «memorice» cada *output* correspondiente a cada *input* (a esta memorización se le llama *overfitting*), puesto que queremos nombres nuevos y originales y no una regurgitación de los que estamos utilizando. Para ello, se han desarrollado un número de técnicas en *deep learning*; pero de momento, utilizaremos una de las más elementales, y consiste en separar nuestros datos en tres partes: entrenamiento, validación y prueba. Entrenaremos nuestro modelo con una cantidad razonable de nombres (el 80 % de ellos), definimos nuestros parámetros —en rigor, hiperparámetros[^3]— con los datos de validación (10 % de nuestros nombres) y verificamos que el modelo sepa generalizar su aprendizaje con los datos destinados a las pruebas (último 10 % de nombres).

In [234]:
import random
random.shuffle(nombres)
n1 = int(0.8*len(nombres))
n2 = int(0.9*len(nombres))

Xtr, Ytr = construir_dataset(nombres[:n1])
Xdev, Ydev = construir_dataset(nombres[n1:n2])
Xte, Yte = construir_dataset(nombres[n2:])
X, Y = construir_dataset(nombres)

X.shape, Xtr.shape, Xdev.shape, Xte.shape

(torch.Size([165469, 3]),
 torch.Size([132468, 3]),
 torch.Size([16441, 3]),
 torch.Size([16560, 3]))

Ahora definimos nuestros hiperparámetros, aunque esta vez utilizaremos $h = 100$:

In [235]:
C = torch.randn(27, 6)
emb = C[Xtr]
h = 100
H = torch.randn((6, h))
d = torch.randn(h)

a = torch.tanh(emb.view(-1, 6) @ H + d)

U = torch.randn(h, 27)
b = torch.randn(27)

logits = a @ U + b

parametros = [C, H, d, U, b]

Nuestro número total de parámetros resultantes es:

In [236]:
sum(p.numel() for p in parametros)

3589

Por otro lado, podemos acelerar el entrenamiento de la red neuronal de la siguiente forma: en cada repetición del *loop* de aprendizaje, podemos seleccionar un «lote» (*batch*) o segmento de los datos para únicamente llevar a cabo el proceso de aprendizaje en ese mismo lote, de manera que el modelo no se entrene tomando siempre en consideración todos los datos a la vez, sino datos —en este caso, nombres— aleatorios en cada iteración. Aunque esto implique sacrificar en cierta medida el desempeño de la red neuronal, sin embargo ese sacrificio se compensa con la rapidez que podemos generar a cambio.

Para crear estos lotes, podemos utilizar el siguiente código, el cual generará índices correspondientes a 32 nombres aleatorios de entre aquellos que pertenezcan a nuestros datos:

In [237]:
ix = torch.randint(0, X.shape[0], (32,))
f'Tres ejemplos: {ix[:3].numpy()}'

'Tres ejemplos: [56193 42924 19114]'

Finalmente, nos falta tratar los detalles técnicos detrás de la *learning rate* («tasa de aprendizaje») del modelo. Recordemos que, cuando modificamos los parámetros en la dirección del gradiente, generalmente atenuamos al gradiente multiplicándolo por un número pequeño para así no excedernos en el ajuste, consiguiendo una pérdida cercana a 0 sin sobrepasarla.

Para determinar una *learning rate* razonable, tenemos que conseguir —empíricamente— los «límites» de la misma:

In [238]:
for p in parametros:
  p.requires_grad = True

In [239]:
X.shape[0]

165469

In [240]:
for _ in range(10):
  #minibatch («minilote»)
  ix = torch.randint(0, X.shape[0], (32,))

  # propagación hacia delante
  emb = C[X[ix]]
  a = torch.tanh(emb.view(-1, 6) @ H + d)
  logits = a @ U + b
  perdida = F.cross_entropy(logits, Y[ix])
  print(perdida.item())
  
  # propagación hacia atrás
  for p in parametros:
    p.grad = None
  perdida.backward()

  # actualización
  lr = -1 # intentaremos obtener el límite superior
  for p in parametros:
    p.data += lr * p.grad

ValueError: ignored

In [ ]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

In [ ]:
plt.plot(lri, lossi)

In [ ]:
10**-0.5

In [ ]:
lri = []
lossi = []
stepi = []

In [ ]:
for i in range(1000):
  #minibatch
  ix = torch.randint(0, Xtr.shape[0], (32,))

  #forward pass
  emb = C[Xtr][ix]
  h = torch.tanh(emb.view(-1, 30) @ W1 + b1)
  logits = h @ W2 + b2
  loss = F.cross_entropy(logits, Ytr[ix])
  
  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()
  #update
  #lr = lrs[i]
  lr = -0.031
  for p in parameters:
    p.data += lr * p.grad

  #track stats
  #lri.append(lre[i])
  stepi.append(i)
  lossi.append(loss.log10().item())

print(loss.item())

In [ ]:
plt.plot(stepi, lossi)

In [ ]:
emb = C[Xdev]
h = torch.tanh(emb.view(-1, 30) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
loss

In [ ]:
block_size = 3
for _ in range(20):
  out = []
  context = [0] * block_size
  while True:
    emb = C[torch.tensor([context])]
    h = torch.tanh(emb.view(1, -1) @ W1 + b1)
    logits = h @ W2 + b2
    probs = F.softmax(logits, dim=1)
    ix = torch.multinomial(probs, num_samples=1).item()
    context = context[1:] + [ix]
    out.append(ix)
    if ix == 0:
      break

  print(''.join(itos[i] for i in out))

[^1]: El método `view` es eficiente porque [no requiere de nuevo espacio](http://blog.ezyang.com/2019/05/pytorch-internals/) en la memoria de nuestra computadora, sino que utiliza el mismo tensor para reacomodarlo de manera distinta. Conforme nuestros programas se vuelvan más complejos, debemos procurar eficientar al máximo nuestros recursos computacionales. Para un acercamiento más general al tema, véase el [artículo de Horace He](https://horace.io/brrr_intro.html).

[^2]: En *deep learning*, el término *hidden* («oculto») se utiliza para referirse a componentes de una red neuronal que no son «visibles» ni como entradas ni como salidas de la red. Se trata de entradas o salidas que se procesan internamente por la red neuronal antes de arrojar un resultado final. La *hidden unit*, en ese sentido, se refiere a cada unidad o componente de una capa oculta (*hidden layer*) en la red. Podemos entenderla como sinónimo de «nodo» o «neurona».

[^3]: Se denominan «parámetros» a los números o coeficientes que la red neuronal aprende y determina durante el entrenamiento (por ejemplo, los números de los pesos y los sesgos). Por otra parte, los «hiperparámetros» son aquellos que nosotros definimos manualmente según la naturaleza de cada problema, y que optimizamos mediante pruebas con distintos valores (por ejemplo, nuestras unidades ocultas $h$).